In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# CSVデータを読み込む（ファイル名は適宜変更）
df = pd.read_csv("../../result.csv")
print(df.columns.tolist())

# グラフの描画
models = df['model'].unique()
datasets = df['dataset'].unique()

for dataset in datasets:
    plt.figure(figsize=(10, 6))
    for model in models:
        for bins in [1, 10]:
            subset = df[(df['model'] == model) & 
                        (df['dataset'] == dataset) & 
                        (df['bins'] == bins)]
            if not subset.empty:
                subset = subset.sort_values('accum_t_ms')
                label = f"{model} (bins={bins})"
                plt.plot(subset['accum_t_ms'], subset['mAP'], marker='o', label=label)

    plt.title(f"Dataset: {dataset}")
    plt.xlabel("Accumulation Time (ms)")
    plt.ylabel("mAP")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{dataset}_mAP_plot.png")
    plt.show()


In [ ]:
for model in df['model'].unique():
    plt.figure(figsize=(10, 6))
    for dataset in df['dataset'].unique():
        for bins in [1, 10]:
            sub = df[(df['model'] == model) & (df['dataset'] == dataset) & (df['bins'] == bins)]
            if not sub.empty:
                sub = sub.sort_values('accum_t_ms')
                label = f"{dataset} (bins={bins})"
                plt.plot(sub['accum_t_ms'], sub['mAP'], marker='o', label=label)
    
    plt.title(f"Accumulation Effect on mAP - Model: {model}")
    plt.xlabel("Accumulation Time (ms)")
    plt.ylabel("mAP")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{model}_accum_effect.png")
    plt.show()


In [ ]:
# 平均mAP（dataset × model × bins）で集計
mean_map = df.groupby(['dataset', 'model', 'bins'])['mAP'].mean().reset_index()

# 棒グラフ描画
for dataset in df['dataset'].unique():
    plt.figure(figsize=(8, 5))
    sub = mean_map[mean_map['dataset'] == dataset]
    for bins in [1, 10]:
        bins_sub = sub[sub['bins'] == bins]
        plt.bar(bins_sub['model'] + f' (bins={bins})', bins_sub['mAP'], label=f"bins={bins}")
    
    plt.title(f"Average mAP per Model (Dataset: {dataset})")
    plt.ylabel("Average mAP")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{dataset}_avg_mAP_bar.png")
    plt.show()
